In [7]:
# Import libraries
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [8]:
# load and view data
data = pd.read_csv('ppp data.csv')
print(f"Shape of the data is: {data.shape}")
data.head()

Shape of the data is: (417266, 15)


,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502.0,111110.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,9.0,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478.0,111110.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/28/2020,Allegiance Bank,TX-18,5


In [9]:
# investigate rows with null values
data[data.isnull().any(axis=1)]

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502.0,111110.0,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,9.0,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478.0,111110.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007.0,111110.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/28/2020,Allegiance Bank,TX-18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417261,"e $150,000-350,000",Lafayette,TX,77064.0,541611.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,8.0,5/1/2020,"JPMorgan Chase Bank, National Association",TX-07,7
417262,"e $150,000-350,000",SANTA ROSA,TX,78205.0,531312.0,Corporation,Unanswered,Male Owned,Unanswered,NaN,18.0,4/14/2020,Vantage Bank Texas,TX-35,7
417263,"e $150,000-350,000",RANCHO CORDOVA,TX,77803.0,442210.0,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,20.0,4/27/2020,"Spirit of Texas Bank, SSB",TX-17,7
417264,"e $150,000-350,000",LOS ALTOS,TX,78230.0,492210.0,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/30/2020,Randolph-Brooks FCU,TX-21,7


In [10]:
# only take rows of businesses that CD is not null

data = data[data['CD'].notna()]
data.head()

#drop rows with NA cities
data = data[data['City'].notna()]

# drop rows with NA NAICS code
data = data[data['NAICSCode'].notna()]

#drop rows with NA business type
data = data[data['BusinessType'].notna()]

data.isnull().sum()

data.shape

data.head()

data['NAICSCode'] = data['NAICSCode'].astype(int)

data

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,LoanRange
0,133500,Pasadena,TX,77502.0,111110,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,"Capital One, National Association",TX-29,6
1,125300,Houston,TX,77041.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,Comerica Bank,TX-07,6
2,124938,PLANO,TX,75024.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,9.0,4/14/2020,Veritex Community Bank,TX-03,5
3,118922,Sugar Land,TX,77478.0,111110,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/27/2020,Allegiance Bank,TX-22,5
4,109952,Houston,TX,77007.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/28/2020,Allegiance Bank,TX-18,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417261,"e $150,000-350,000",Lafayette,TX,77064.0,541611,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,8.0,5/1/2020,"JPMorgan Chase Bank, National Association",TX-07,7
417262,"e $150,000-350,000",SANTA ROSA,TX,78205.0,531312,Corporation,Unanswered,Male Owned,Unanswered,NaN,18.0,4/14/2020,Vantage Bank Texas,TX-35,7
417263,"e $150,000-350,000",RANCHO CORDOVA,TX,77803.0,442210,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,20.0,4/27/2020,"Spirit of Texas Bank, SSB",TX-17,7
417264,"e $150,000-350,000",LOS ALTOS,TX,78230.0,492210,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/30/2020,Randolph-Brooks FCU,TX-21,7


In [11]:
## read in data for NAICS code

naics = pd.read_csv('US-Business-Profiles-By-Sales-and-Employees (2).csv')

naics.head()

,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming


In [17]:
naics.dtypes
naics[naics.isnull().any(axis=1)]


naics['NAICS 1 Code'] = naics['NAICS 1 Code'].astype(int)

naics

,NAICS 1 Code,NAICS 1 Description
0,111110,Soybean Farming
1,111120,Oilseed (except Soybean) Farming
2,111130,Dry Pea and Bean Farming
3,111140,Wheat Farming
4,111150,Corn Farming
...,...,...
1040,926140,Regulation of Agricultural Marketing and Commo...
1041,926150,"Regulation, Licensing, and Inspection of Misce..."
1042,927110,Space Research and Technology
1043,928110,National Security


In [22]:
## merge tables 

loan_data = pd.merge(data, naics, left_on='NAICSCode', right_on='NAICS 1 Code')
loan_data

,LoanAmount,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsReported,DateApproved,Lender,CD,LoanRange,NAICS 1 Code,NAICS 1 Description
0,133500,Pasadena,TX,77502.0,111110,Subchapter S Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,"Capital One, National Association",TX-29,6,111110,Soybean Farming
1,125300,Houston,TX,77041.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,0.0,4/29/2020,Comerica Bank,TX-07,6,111110,Soybean Farming
2,124938,PLANO,TX,75024.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,9.0,4/14/2020,Veritex Community Bank,TX-03,5,111110,Soybean Farming
3,118922,Sugar Land,TX,77478.0,111110,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/27/2020,Allegiance Bank,TX-22,5,111110,Soybean Farming
4,109952,Houston,TX,77007.0,111110,Corporation,Unanswered,Unanswered,Unanswered,NaN,17.0,4/28/2020,Allegiance Bank,TX-18,5,111110,Soybean Farming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397943,"d $350,000-1 million",PHOENIX,TX,79072.0,325193,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,63.0,4/15/2020,"JPMorgan Chase Bank, National Association",TX-19,8,325193,Ethyl Alcohol Manufacturing
397944,"d $350,000-1 million",TUCSON,TX,79045.0,325193,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,44.0,4/15/2020,"JPMorgan Chase Bank, National Association",TX-19,8,325193,Ethyl Alcohol Manufacturing
397945,"d $350,000-1 million",Scottsdale,TX,75034.0,325193,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,25.0,4/10/2020,"JPMorgan Chase Bank, National Association",TX-26,8,325193,Ethyl Alcohol Manufacturing
397946,"d $350,000-1 million",YUMA,TX,77002.0,212113,Limited Liability Company(LLC),Unanswered,Unanswered,Unanswered,NaN,15.0,4/9/2020,Gulf Capital Bank,TX-18,8,212113,Anthracite Mining


In [49]:
# drop lenders with loans of 5 or more
counts = loan_data['Lender'].value_counts()

loan_data_top = loan_data.loc[loan_data['Lender'].isin(counts.index[counts > 5])]
loan_data_top.shape

(396227, 17)

In [50]:
# Assign X (data) and y(target)
X = loan_data_top.drop("Lender", axis=1)
y = loan_data_top["Lender"]
print(X.shape, y.shape)

(396227, 16) (396227,)


In [51]:
# Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [52]:
# Create a logistic Regression Model
classifier = LogisticRegression()
classifier

LogisticRegression()

In [53]:
# Fit model using the training data
classifier.fit(X_train, y_train)

ValueError: could not convert string to float: 'Civic and Social Organizations'

In [ ]:
# evaluate the model's performance using the test data
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

In [ ]:
# Make predictions
predictions = classifier.predict(X_test)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

In [ ]:
# View predictions along with their probabilities of being within each class 
##  this will be the difficult part of our project because data is not binary like example: male/female
probs = classifier.predict_proba(X_test)
pred_df = pd.DataFrame({
    "Prediction": predictions, 
    "Actual": y_test, 
    "P(Female)": [p[0] for p in probs], 
    "P(Male)": [p[1] for p in probs]
    }).reset_index(drop=True)

pred_df.head(10)

In [ ]:
# Model Evaluation
# Create a Confusion Matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predictions)

In [ ]:
# Label the confusion Matrix
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Neg: {tn}   False Pos: {fp}")
print(f"False Neg: {fn}  True Pos: {tp}")

In [ ]:
# Plot confusion matrix
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(classifier, X_test, y_test, cmap="Blues")

In [ ]:
# Precision and Recall
### we don't technically have a "positive" label, like we would if we were classifying emails as Spam, or patients as having cancer. 
## This means that we need to tell our precision and recall functions which class label we want to consider to be our "positive" class i.e. which lender
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
## Random tests of lenders first test is JPMorgan Chase Bank, National Association, the lender with the most loans
precision_score(y_test, predictions, pos_label="JPMorgan Chase Bank, National Association")

In [ ]:
## Random tests of lenders second test is First National Bank of Louisiana, random test of lender with lower loan count of 10 loans
precision_score(y_test, predictions, pos_label="First National Bank of Louisiana")

In [ ]:
#  what is the harmonic mean of JP Morgan from first test
f1_score(y_test, predictions, pos_label="JPMorgan Chase Bank, National Association")

In [ ]:
#  what is the harmonic mean of First National Bank of Louisiana from second test
f1_score(y_test, predictions, pos_label="First National Bank of Louisiana")